# Assignment 1
## This notebook implements analysis of a number of ML models using two specific datasets
These two datasets look at Diabetes health predictors and Airline Passenger Satisfaction.

### Airline Passenger 

In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import LearningCurveDisplay, ShuffleSplit
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate
from sklearn.model_selection import learning_curve
from sklearn.model_selection import cross_val_score
from sklearn.metrics import fbeta_score, make_scorer
import matplotlib.pyplot as plt

In [4]:
airline_data = pd.read_csv('AirlinePassengerSatisfaction/train.csv',index_col=0)

In [5]:

diabetes_data =  pd.read_csv('DiabetesHealthIndicatorDataset/diabetes_binary_health_indicators_BRFSS2015.csv')



In [6]:
airline_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103904 entries, 0 to 103903
Data columns (total 24 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id                                 103904 non-null  int64  
 1   Gender                             103904 non-null  object 
 2   Customer Type                      103904 non-null  object 
 3   Age                                103904 non-null  int64  
 4   Type of Travel                     103904 non-null  object 
 5   Class                              103904 non-null  object 
 6   Flight Distance                    103904 non-null  int64  
 7   Inflight wifi service              103904 non-null  int64  
 8   Departure/Arrival time convenient  103904 non-null  int64  
 9   Ease of Online booking             103904 non-null  int64  
 10  Gate location                      103904 non-null  int64  
 11  Food and drink                     1039

In [13]:
airline_data.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [14]:
airline_data = airline_data.loc[:, airline_data.columns!='id']

In [15]:

airline_data.boxplot()

<AxesSubplot: >

In [16]:

fig = airline_data.hist()


In [17]:
# split the dataframe into its numerical and categorical components
X_num = airline_data.select_dtypes(exclude='object')
X_cat = airline_data.select_dtypes(include='object')

In [18]:
X_num

,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,13,460,3,4,3,1,5,3,5,5,4,3,4,4,5,5,25,18.0
1,25,235,3,2,3,3,1,3,1,1,1,5,3,1,4,1,1,6.0
2,26,1142,2,2,2,2,5,5,5,5,4,3,4,4,4,5,0,0.0
3,25,562,2,5,5,5,2,2,2,2,2,5,3,1,4,2,11,9.0
4,61,214,3,3,3,3,4,5,5,3,3,4,4,3,3,3,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,23,192,2,1,2,3,2,2,2,2,3,1,4,2,3,2,3,0.0
103900,49,2347,4,4,4,4,2,4,5,5,5,5,5,5,5,4,0,0.0
103901,30,1995,1,1,1,3,4,1,5,4,3,2,4,5,5,4,7,14.0
103902,22,1000,1,1,1,5,1,1,1,1,4,5,1,5,4,1,0,0.0


In [19]:
X_cat

,Gender,Customer Type,Type of Travel,Class,satisfaction
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied
2,Female,Loyal Customer,Business travel,Business,satisfied
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
4,Male,Loyal Customer,Business travel,Business,satisfied
...,...,...,...,...,...
103899,Female,disloyal Customer,Business travel,Eco,neutral or dissatisfied
103900,Male,Loyal Customer,Business travel,Business,satisfied
103901,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied
103902,Female,disloyal Customer,Business travel,Eco,neutral or dissatisfied


In [20]:
label_encoder = LabelEncoder()

In [21]:
X_cat.apply(label_encoder.fit_transform)

,Gender,Customer Type,Type of Travel,Class,satisfaction
0,1,0,1,2,0
1,1,1,0,0,0
2,0,0,0,0,1
3,0,0,0,0,0
4,1,0,0,0,1
...,...,...,...,...,...
103899,0,1,0,1,0
103900,1,0,0,0,1
103901,1,1,0,0,0
103902,0,1,0,1,0


In [22]:
from collections import defaultdict
d = defaultdict(LabelEncoder)

In [23]:
fit = X_cat.apply(lambda x: d[x.name].fit_transform(x))

In [24]:
fit

,Gender,Customer Type,Type of Travel,Class,satisfaction
0,1,0,1,2,0
1,1,1,0,0,0
2,0,0,0,0,1
3,0,0,0,0,0
4,1,0,0,0,1
...,...,...,...,...,...
103899,0,1,0,1,0
103900,1,0,0,0,1
103901,1,1,0,0,0
103902,0,1,0,1,0


In [25]:
fit.apply(lambda x: d[x.name].inverse_transform(x))

,Gender,Customer Type,Type of Travel,Class,satisfaction
0,Male,Loyal Customer,Personal Travel,Eco Plus,neutral or dissatisfied
1,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied
2,Female,Loyal Customer,Business travel,Business,satisfied
3,Female,Loyal Customer,Business travel,Business,neutral or dissatisfied
4,Male,Loyal Customer,Business travel,Business,satisfied
...,...,...,...,...,...
103899,Female,disloyal Customer,Business travel,Eco,neutral or dissatisfied
103900,Male,Loyal Customer,Business travel,Business,satisfied
103901,Male,disloyal Customer,Business travel,Business,neutral or dissatisfied
103902,Female,disloyal Customer,Business travel,Eco,neutral or dissatisfied


In [26]:
enc = OrdinalEncoder(categories=[['neutral or dissatisfied','satisfied']])

In [27]:
enc.fit_transform(X_cat[['satisfaction']])

array([[0.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [0.]])

In [28]:
enc = OrdinalEncoder(categories=[['Eco','Eco Plus','Business']])

In [29]:
enc.fit_transform(X_cat[['Class']])

array([[1.],
       [2.],
       [2.],
       ...,
       [2.],
       [0.],
       [2.]])

In [30]:
enc = OrdinalEncoder(categories=[['disloyal Customer','Loyal Customer']])

In [31]:
enc.fit_transform(X_cat[['Customer Type']])

array([[1.],
       [0.],
       [1.],
       ...,
       [0.],
       [0.],
       [1.]])

In [32]:
airline_data_encoded = airline_data.copy()

In [33]:
airline_data_encoded.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [34]:
enc = OrdinalEncoder(categories=[['neutral or dissatisfied','satisfied']])

In [35]:

airline_data_encoded['satisfaction'] = enc.fit_transform(airline_data_encoded[['satisfaction']])


In [36]:
airline_data_encoded.head(10)

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0.0
1,Male,disloyal Customer,25,Business travel,Business,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0.0
2,Female,Loyal Customer,26,Business travel,Business,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1.0
3,Female,Loyal Customer,25,Business travel,Business,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0.0
4,Male,Loyal Customer,61,Business travel,Business,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1.0
5,Female,Loyal Customer,26,Personal Travel,Eco,1180,3,4,2,1,...,1,3,4,4,4,4,1,0,0.0,0.0
6,Male,Loyal Customer,47,Personal Travel,Eco,1276,2,4,2,3,...,2,3,3,4,3,5,2,9,23.0,0.0
7,Female,Loyal Customer,52,Business travel,Business,2035,4,3,4,4,...,5,5,5,5,4,5,4,4,0.0,1.0
8,Female,Loyal Customer,41,Business travel,Business,853,1,2,2,2,...,1,1,2,1,4,1,2,0,0.0,0.0
9,Male,disloyal Customer,20,Business travel,Eco,1061,3,3,3,4,...,2,2,3,4,4,3,2,0,0.0,0.0


In [37]:
enc = OrdinalEncoder(categories=[['Eco','Eco Plus','Business']])

In [38]:
airline_data_encoded['Class'] = enc.fit_transform(airline_data_encoded[['Class']])

In [39]:
airline_data_encoded.head(10)

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,Loyal Customer,13,Personal Travel,1.0,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0.0
1,Male,disloyal Customer,25,Business travel,2.0,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0.0
2,Female,Loyal Customer,26,Business travel,2.0,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1.0
3,Female,Loyal Customer,25,Business travel,2.0,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0.0
4,Male,Loyal Customer,61,Business travel,2.0,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1.0
5,Female,Loyal Customer,26,Personal Travel,0.0,1180,3,4,2,1,...,1,3,4,4,4,4,1,0,0.0,0.0
6,Male,Loyal Customer,47,Personal Travel,0.0,1276,2,4,2,3,...,2,3,3,4,3,5,2,9,23.0,0.0
7,Female,Loyal Customer,52,Business travel,2.0,2035,4,3,4,4,...,5,5,5,5,4,5,4,4,0.0,1.0
8,Female,Loyal Customer,41,Business travel,2.0,853,1,2,2,2,...,1,1,2,1,4,1,2,0,0.0,0.0
9,Male,disloyal Customer,20,Business travel,0.0,1061,3,3,3,4,...,2,2,3,4,4,3,2,0,0.0,0.0


In [40]:
enc = OrdinalEncoder(categories=[['disloyal Customer','Loyal Customer']])

In [41]:

airline_data_encoded['Customer Type'] = enc.fit_transform(airline_data_encoded[['Customer Type']])


In [42]:
airline_data_encoded.head(10)

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,Male,1.0,13,Personal Travel,1.0,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0.0
1,Male,0.0,25,Business travel,2.0,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0.0
2,Female,1.0,26,Business travel,2.0,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1.0
3,Female,1.0,25,Business travel,2.0,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0.0
4,Male,1.0,61,Business travel,2.0,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1.0
5,Female,1.0,26,Personal Travel,0.0,1180,3,4,2,1,...,1,3,4,4,4,4,1,0,0.0,0.0
6,Male,1.0,47,Personal Travel,0.0,1276,2,4,2,3,...,2,3,3,4,3,5,2,9,23.0,0.0
7,Female,1.0,52,Business travel,2.0,2035,4,3,4,4,...,5,5,5,5,4,5,4,4,0.0,1.0
8,Female,1.0,41,Business travel,2.0,853,1,2,2,2,...,1,1,2,1,4,1,2,0,0.0,0.0
9,Male,0.0,20,Business travel,0.0,1061,3,3,3,4,...,2,2,3,4,4,3,2,0,0.0,0.0


In [43]:
label_encoder = LabelEncoder()

In [44]:
airline_data_encoded['Gender'] = label_encoder.fit_transform(airline_data_encoded['Gender'])

In [45]:
airline_data_encoded.head(10)

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,1.0,13,Personal Travel,1.0,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0.0
1,1,0.0,25,Business travel,2.0,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0.0
2,0,1.0,26,Business travel,2.0,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1.0
3,0,1.0,25,Business travel,2.0,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0.0
4,1,1.0,61,Business travel,2.0,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1.0
5,0,1.0,26,Personal Travel,0.0,1180,3,4,2,1,...,1,3,4,4,4,4,1,0,0.0,0.0
6,1,1.0,47,Personal Travel,0.0,1276,2,4,2,3,...,2,3,3,4,3,5,2,9,23.0,0.0
7,0,1.0,52,Business travel,2.0,2035,4,3,4,4,...,5,5,5,5,4,5,4,4,0.0,1.0
8,0,1.0,41,Business travel,2.0,853,1,2,2,2,...,1,1,2,1,4,1,2,0,0.0,0.0
9,1,0.0,20,Business travel,0.0,1061,3,3,3,4,...,2,2,3,4,4,3,2,0,0.0,0.0


In [46]:
airline_data_encoded['Type of Travel'] = label_encoder.fit_transform(airline_data_encoded['Type of Travel'])

In [47]:
airline_data_encoded.head(10)

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,1.0,13,1,1.0,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0.0
1,1,0.0,25,0,2.0,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0.0
2,0,1.0,26,0,2.0,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1.0
3,0,1.0,25,0,2.0,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0.0
4,1,1.0,61,0,2.0,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1.0
5,0,1.0,26,1,0.0,1180,3,4,2,1,...,1,3,4,4,4,4,1,0,0.0,0.0
6,1,1.0,47,1,0.0,1276,2,4,2,3,...,2,3,3,4,3,5,2,9,23.0,0.0
7,0,1.0,52,0,2.0,2035,4,3,4,4,...,5,5,5,5,4,5,4,4,0.0,1.0
8,0,1.0,41,0,2.0,853,1,2,2,2,...,1,1,2,1,4,1,2,0,0.0,0.0
9,1,0.0,20,0,0.0,1061,3,3,3,4,...,2,2,3,4,4,3,2,0,0.0,0.0


In [48]:
#airline_data_encoded.sort_values('id').set_index('id')

In [49]:
# https://www.section.io/engineering-education/how-to-implement-k-fold-cross-validation/
def cross_validation(model, _X, _y, _cv=5):
    '''Function to perform 5 Folds Cross-Validation
       Parameters
       ----------
      model: Python Class, default=None
              This is the machine learning algorithm to be used for training.
      _X: array
           This is the matrix of features.
      _y: array
           This is the target variable.
      _cv: int, default=5
          Determines the number of folds for cross-validation.
       Returns
       -------
       The function returns a dictionary containing the metrics 'accuracy', 'precision',
       'recall', 'f1' for both training set and validation set.
      '''
    _scoring = ['accuracy', 'precision', 'recall', 'f1']
    results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
    return results
      # return {"Training Accuracy scores": results['train_accuracy'],
      #         "Mean Training Accuracy": results['train_accuracy'].mean()*100,
      #         "Training Precision scores": results['train_precision'],
      #         "Mean Training Precision": results['train_precision'].mean(),
      #         "Training Recall scores": results['train_recall'],
      #         "Mean Training Recall": results['train_recall'].mean(),
      #         "Training F1 scores": results['train_f1'],
      #         "Mean Training F1 Score": results['train_f1'].mean(),
      #         "Validation Accuracy scores": results['test_accuracy'],
      #         "Mean Validation Accuracy": results['test_accuracy'].mean()*100,
      #         "Validation Precision scores": results['test_precision'],
      #         "Mean Validation Precision": results['test_precision'].mean(),
      #         "Validation Recall scores": results['test_recall'],
      #         "Mean Validation Recall": results['test_recall'].mean(),
      #         "Validation F1 scores": results['test_f1'],
      #         "Mean Validation F1 Score": results['test_f1'].mean()
      #         }

In [50]:
airline_data_encoded.columns

Index(['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes',
       'satisfaction'],
      dtype='object')

In [51]:
airline_data_encoded.isna().sum().sum()

310

In [52]:
airline_data_encoded = airline_data_encoded[~airline_data_encoded.isna().any(axis=1)]

In [53]:
airline_data_encoded

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,1,1.0,13,1,1.0,460,3,4,3,1,...,5,4,3,4,4,5,5,25,18.0,0.0
1,1,0.0,25,0,2.0,235,3,2,3,3,...,1,1,5,3,1,4,1,1,6.0,0.0
2,0,1.0,26,0,2.0,1142,2,2,2,2,...,5,4,3,4,4,4,5,0,0.0,1.0
3,0,1.0,25,0,2.0,562,2,5,5,5,...,2,2,5,3,1,4,2,11,9.0,0.0
4,1,1.0,61,0,2.0,214,3,3,3,3,...,3,3,4,4,3,3,3,0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,0,0.0,23,0,0.0,192,2,1,2,3,...,2,3,1,4,2,3,2,3,0.0,0.0
103900,1,1.0,49,0,2.0,2347,4,4,4,4,...,5,5,5,5,5,5,4,0,0.0,1.0
103901,1,0.0,30,0,2.0,1995,1,1,1,3,...,4,3,2,4,5,5,4,7,14.0,0.0
103902,0,0.0,22,0,0.0,1000,1,1,1,5,...,1,4,5,1,5,4,1,0,0.0,0.0


In [54]:
X = airline_data_encoded[['Gender', 'Customer Type', 'Age', 'Type of Travel', 'Class',
       'Flight Distance', 'Inflight wifi service',
       'Departure/Arrival time convenient', 'Ease of Online booking',
       'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort',
       'Inflight entertainment', 'On-board service', 'Leg room service',
       'Baggage handling', 'Checkin service', 'Inflight service',
       'Cleanliness', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']]
y = airline_data_encoded['satisfaction']

In [55]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = X.copy()
X_train_minmax[X_train_minmax.columns] = min_max_scaler.fit_transform(X_train_minmax[X_train_minmax.columns])

In [56]:
X_train_minmax

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,...,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes
0,1.0,1.0,0.076923,1.0,0.5,0.086632,0.6,0.8,0.6,0.2,...,1.0,1.0,0.8,0.6,0.75,0.8,1.0,1.0,0.015704,0.011364
1,1.0,0.0,0.230769,0.0,1.0,0.041195,0.6,0.4,0.6,0.6,...,0.2,0.2,0.2,1.0,0.50,0.2,0.8,0.2,0.000628,0.003788
2,0.0,1.0,0.243590,0.0,1.0,0.224354,0.4,0.4,0.4,0.4,...,1.0,1.0,0.8,0.6,0.75,0.8,0.8,1.0,0.000000,0.000000
3,0.0,1.0,0.230769,0.0,1.0,0.107229,0.4,1.0,1.0,1.0,...,0.4,0.4,0.4,1.0,0.50,0.2,0.8,0.4,0.006910,0.005682
4,1.0,1.0,0.692308,0.0,1.0,0.036955,0.6,0.6,0.6,0.6,...,1.0,0.6,0.6,0.8,0.75,0.6,0.6,0.6,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103899,0.0,0.0,0.205128,0.0,0.0,0.032512,0.4,0.2,0.4,0.6,...,0.4,0.4,0.6,0.2,0.75,0.4,0.6,0.4,0.001884,0.000000
103900,1.0,1.0,0.538462,0.0,1.0,0.467690,0.8,0.8,0.8,0.8,...,1.0,1.0,1.0,1.0,1.00,1.0,1.0,0.8,0.000000,0.000000
103901,1.0,0.0,0.294872,0.0,1.0,0.396607,0.2,0.2,0.2,0.6,...,1.0,0.8,0.6,0.4,0.75,1.0,1.0,0.8,0.004397,0.008838
103902,0.0,0.0,0.192308,0.0,0.0,0.195679,0.2,0.2,0.2,1.0,...,0.2,0.2,0.8,1.0,0.00,1.0,0.8,0.2,0.000000,0.000000


In [57]:
%%time
# decision tree models
decision_tree_model = DecisionTreeClassifier(criterion="log_loss",
                                     random_state=0)
# decision_tree_result = cross_validation(decision_tree_model, X, y, 5)
# print(decision_tree_result)


CPU times: user 22 µs, sys: 1 µs, total: 23 µs
Wall time: 26 µs


In [58]:
%%time
# k nearest neighbors
knn_model = KNeighborsClassifier(n_neighbors=3,p=2)
# knn_result = cross_validation(knn_model, X, y, 5)
# print(knn_result)


CPU times: user 23 µs, sys: 1 µs, total: 24 µs
Wall time: 27.9 µs


In [59]:
%%time
knn_model = KNeighborsClassifier(n_neighbors=5,weights='uniform', p=1, algorithm='auto')
# knn_result = cross_validation(knn_model, X, y, 5)
# print(knn_result)

CPU times: user 14 µs, sys: 1 µs, total: 15 µs
Wall time: 20.3 µs


In [ ]:
%%time
#suppot vector machine
svm_model = svm.SVC(random_state=0)

In [ ]:
%%time
#neural network performance models - perceptron
nn_model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                         hidden_layer_sizes=(5, 2), random_state=1)

In [60]:
train_sizes_range = np.linspace(0.01,1.0,200)

In [ ]:
%%time
train_sizes, train_scores, validation_scores = \
    learning_curve(estimator = decision_tree_model,
                   X = X_train_minmax,
                   y = y, train_sizes = train_sizes_range, cv = 5,
                   scoring = 'accuracy') #['accuracy', 'precision', 'recall', 'f1'])

In [61]:
%%time

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 6), sharey=True)
naive_bayes = GaussianNB()
svc = SVC(kernel="rbf", gamma=0.001)
common_params = {
    "X": X_train_minmax,
    "y": y,
    "train_sizes": np.linspace(0.1, 1.0, 5),
    "cv": ShuffleSplit(n_splits=50, test_size=0.2, random_state=0),
    "score_type": "both",
    "n_jobs": -1,
    "line_kw": {"marker": "o"},
    "std_display_style": "fill_between",
    "score_name": "Accuracy",
}

for ax_idx, estimator in enumerate([naive_bayes, svc]):
    disp = LearningCurveDisplay.from_estimator(estimator, **common_params, ax=ax[ax_idx])
    disp.plot()
    handles, label = ax[ax_idx].get_legend_handles_labels()
    ax[ax_idx].legend(handles[:2], ["Training Score", "Test Score"])
    ax[ax_idx].set_title(f"Learning Curve for {estimator.__class__.__name__}")

/Users/rrj/miniforge3/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
breaker

# train_scores_mean = train_scores.mean(axis = 1)
# validation_scores_mean = validation_scores.mean(axis = 1)
# print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
# print('\n', '-' * 20) # separator
# print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))

In [ ]:


# #plt.style.use('seaborn')
# plt.plot(train_sizes, train_scores_mean, label = 'Training error')
# plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
# plt.ylabel('MSE', fontsize = 14)
# plt.xlabel('Training set size', fontsize = 14)
# plt.title('Learning curves for Decision Tree Model', fontsize = 18, y = 1.03)
# plt.legend()
# plt.ylim(0,1)

In [ ]:
#
# Calculate training and test mean and std
#
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(validation_scores, axis=1)
test_std = np.std(validation_scores, axis=1)
#
# Plot the learning curve
#
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()

In [ ]:
%%time
train_sizes, train_scores, validation_scores = \
    learning_curve(estimator = knn_model,
                   X = X.iloc[:5000],
                   y = y.iloc[:5000], train_sizes = train_sizes, cv = 5,
                   scoring = 'accuracy') #['accuracy', 'precision', 'recall', 'f1'])

In [ ]:
#plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training error')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for KNN Model', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1)

In [ ]:
svm_model = svm.SVC(random_state=0)

In [ ]:
%%time
train_sizes, train_scores, validation_scores = \
    learning_curve(estimator = svm_model,
                   X = X.iloc[:5000],
                   y = y.iloc[:5000], train_sizes = train_sizes, cv = 5,
                   scoring = 'accuracy') #['accuracy', 'precision', 'recall', 'f1'])

In [ ]:
#plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training error')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for SVM', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1)

In [ ]:
X.iloc[:5000]

In [ ]:
nn_model = MLPClassifier(solver='lbfgs', alpha=1e-5,
                         hidden_layer_sizes=(5, 2), random_state=1)

In [ ]:
min_max_scaler = preprocessing.MinMaxScaler()
X_train_minmax = X.copy()
X_train_minmax[X_train_minmax.columns] = min_max_scaler.fit_transform(X_train_minmax[X_train_minmax.columns])


In [ ]:
%%time
# train_sizes, train_scores, validation_scores = \
#     learning_curve(estimator = nn_model,
#                    X = X_train_minmax.iloc[:5000],
#                    y = y.iloc[:5000], train_sizes = train_sizes, cv = 5,
#                    scoring = 'accuracy') #['accuracy', 'precision', 'recall', 'f1'])

In [ ]:
#plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training error')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for SVM', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1)

In [ ]:
X_train_minmax

In [ ]:
X

In [ ]:
y.sum()/len(y)

### Now Diabetes Dataset

In [ ]:

diabetes_data =  pd.read_csv('DiabetesHealthIndicatorDataset/diabetes_binary_health_indicators_BRFSS2015.csv')



In [ ]:
diabetes_data.info()

In [ ]:
diabetes_data.boxplot()

In [ ]:
plt.xticks(fontsize=6)
diabetes_data.hist()

In [ ]:
diabetes_data['Diabetes_binary'].sum()/len(diabetes_data['Diabetes_binary'])

In [ ]:
diabetes_data['Diabetes_binary'].value_counts()

In [ ]:
diabetes_data =  pd.read_csv('DiabetesHealthIndicatorDataset/diabetes_binary_health_indicators_BRFSS2015.csv')


In [ ]:


diabetes_train, diabetest_test = train_test_split(diabetes_data, test_size=0.2)

In [ ]:
ros = RandomOverSampler(random_state=0)
X = diabetes_train.loc[:, diabetes_train.columns!='Diabetes_binary']
y = diabetes_train['Diabetes_binary']
X_resampled, y_resampled = ros.fit_resample(X,y)

In [ ]:
y

In [ ]:
X_resampled

In [ ]:
y_resampled.value_counts()

In [ ]:
rus = RandomUnderSampler(random_state=0)

In [ ]:
X_resampled, y_resampled = rus.fit_resample(X,y)

In [ ]:
y_resampled.value_counts()

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled , test_size=0.33, random_state=42)


In [ ]:
len(X_train)

In [ ]:
%%time
decision_tree_model = DecisionTreeClassifier(criterion="entropy",
                                     random_state=0)
decision_tree_result = cross_validation(decision_tree_model, X_train, y_train, 5)
print(decision_tree_result)

In [ ]:
%%time
train_sizes, train_scores, validation_scores = \
    learning_curve(estimator = decision_tree_model,
                   X = X_train,
                   y = y_train, train_sizes = train_sizes_range, cv = 5,
                   scoring = 'accuracy') #['accuracy', 'precision', 'recall', 'f1'])

In [ ]:
train_scores_mean = train_scores.mean(axis = 1)
validation_scores_mean = validation_scores.mean(axis = 1)
print('Mean training scores\n\n', pd.Series(train_scores_mean, index = train_sizes))
print('\n', '-' * 20) # separator
print('\nMean validation scores\n\n',pd.Series(validation_scores_mean, index = train_sizes))

In [ ]:
#plt.style.use('seaborn')
plt.plot(train_sizes, train_scores_mean, label = 'Training error')
plt.plot(train_sizes, validation_scores_mean, label = 'Validation error')
plt.ylabel('Accuracy', fontsize = 14)
plt.xlabel('Training set size', fontsize = 14)
plt.title('Learning curves for Decision Tree Diabetes', fontsize = 18, y = 1.03)
plt.legend()
plt.ylim(0,1)

In [ ]:
#
# Calculate training and test mean and std
#
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
test_mean = np.mean(validation_scores, axis=1)
test_std = np.std(validation_scores, axis=1)
#
# Plot the learning curve
#
plt.plot(train_sizes, train_mean, color='blue', marker='o', markersize=5, label='Training Accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean - train_std, alpha=0.15, color='blue')
plt.plot(train_sizes, test_mean, color='green', marker='+', markersize=5, linestyle='--', label='Validation Accuracy')
plt.fill_between(train_sizes, test_mean + test_std, test_mean - test_std, alpha=0.15, color='green')
plt.title('Learning Curve')
plt.xlabel('Training Data Size')
plt.ylabel('Model accuracy')
plt.grid()
plt.legend(loc='lower right')
plt.show()

In [ ]:

params = {'max_leaf_nodes': list(range(20, 200)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(decision_tree_model, params, verbose=1,n_jobs=-1, cv=3,return_train_score=True)
grid_search_cv.fit(X_train, y_train)

In [ ]:
grid_search_cv.best_estimator_

In [ ]:
grid_search_cv.cv_results_